# Using ZeroEntropy as a Search Tool for Voice AI Agents

This cookbook shows how to build a voice assistant that can search through YC company data using ZeroEntropy's MCP server.

## What We'll Build

A voice assistant that:
- 🎤 Records your voice using Enter key
- 🔍 Searches YC company database via ZeroEntropy
- 🤖 Responds with relevant information
- ⏸️ Can be interrupted during responses

## Prerequisites

- Python 3.8+
- ZeroEntropy API key ([Get one here](https://dashboard.zeroentropy.dev))
- OpenAI API key
- Microphone access


## Step 1: Install Dependencies

First, let's install all the required packages:

In [ ]:
%pip install openai zeroentropy requests openai-agents sounddevice numpy python-dotenv

## Step 2: Configuration & Setup

Create a `.env` file in this directory with your API keys:
```
ZEROENTROPY_API_KEY=your_zeroentropy_key_here
OPENAI_API_KEY=your_openai_key_here
```

Now let's load configuration and import everything we need:

In [ ]:
import os
import threading
import time
import io
import sys
import select

import dotenv
import requests
import sounddevice as sd
import numpy as np
from openai import OpenAI
from zeroentropy import ZeroEntropy
from agents import Agent
from agents.mcp import MCPServerSse
from agents.voice import (
    AudioInput,
    SingleAgentVoiceWorkflow,
    VoicePipeline,
    VoicePipelineConfig,
    TTSModelSettings,
)

# Load environment variables
dotenv.load_dotenv()

# Configuration
ZEROENTROPY_API_KEY = os.getenv("ZEROENTROPY_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COLLECTION_NAME = "yc_voice_agent_support"
MCP_URL = "https://openai-deepresearch.zeroentropy.dev/sse/"
YC_API_URL = "https://yc-oss.github.io/api/companies/all.json"

# Audio settings
SAMPLE_RATE = 24000
CHANNELS = 1

# AI prompts
SYSTEM_PROMPT = (
    "You receive transcribed user speech. "
    "Rewrite the query into a precise search, always call the search tool via MCP, "
    "fetch results, and answer in English, succinctly."
)

TTS_PROMPT = (
    "You are a helpful voice assistant who can answer any question about any YC company. "
    "Personality: Helpful and concise voice assistant. "
    "Tone: Friendly and clear. "
    "Pause naturally between points."
)

# Initialize clients
ze_client = ZeroEntropy(api_key=ZEROENTROPY_API_KEY)
openai_client = OpenAI(api_key=OPENAI_API_KEY)

print("✅ Configuration loaded successfully!")

## Step 3: Create Collection & Load YC Data

We'll create a ZeroEntropy collection and populate it with Y Combinator company data. This will take a few minutes to complete.

In [ ]:
# Create collection
try:
    ze_client.collections.add(collection_name=COLLECTION_NAME)
    print(f"✅ Created collection: {COLLECTION_NAME}")
except Exception as e:
    print(f"⚠️ Collection already exists: {e}")

# Function to load YC company data
def setup_yc_data():
    """Fetch YC companies and add to ZeroEntropy collection."""
    print("🔄 Fetching YC company data...")
    
    try:
        response = requests.get(YC_API_URL, timeout=30)
        response.raise_for_status()
        companies = response.json()
        print(f"📥 Fetched {len(companies)} companies")
    except Exception as e:
        print(f"❌ Failed to fetch companies: {e}")
        return False
    
    # Add companies to collection
    success_count = 0
    for company in companies:
        try:
            slug = str(company.get('slug', ''))
            text = (
                f"{company.get('name', '')} — {company.get('one_liner', '')}\n\n"
                f"{company.get('long_description', '')}\n\n"
                f"{company.get('website', '')}\n\n"
                f"{company.get('subindustry', '')}\n\n"
                f"Stage: {company.get('stage', '')}"
            )
            metadata = {
                "batch": company.get("batch", ""),
                "list:industries": company.get("industries", []),
                "stage": company.get("stage", ""),
            }
            
            ze_client.documents.add(
                collection_name=COLLECTION_NAME,
                path=slug,
                content={"type": "text", "text": text},
                metadata=metadata
            )
            success_count += 1
        except Exception:
            continue
    
    print(f"✅ Added {success_count} companies to collection")
    return success_count > 0

# Run the setup
if setup_yc_data():
    print("🎉 YC data setup complete!")


## Step 4: Audio Functions

These functions handle recording your voice and transcribing it to text. We use the Enter key instead of space bar to avoid macOS accessibility permission issues.


In [1]:
def record_enter_to_talk():
    """Record audio until Enter key is pressed."""
    print("Press ENTER to start recording, then press ENTER again to stop...")
    
    # Wait for first Enter press to start recording
    input("Press ENTER to start recording: ")
    print("🎤 Recording... Press ENTER to stop")
    
    audio_data = []
    recording = True
    
    def audio_callback(indata, frames, time, status):
        if recording:
            audio_data.append(indata.copy())
    
    # Start audio recording
    stream = sd.InputStream(
        samplerate=SAMPLE_RATE,
        channels=CHANNELS,
        dtype="int16",
        callback=audio_callback
    )
    
    with stream:
        # Wait for second Enter press to stop recording
        input()  # This will block until Enter is pressed
        recording = False
        print("⏹️ Recording stopped")
    
    if audio_data:
        return np.concatenate(audio_data, axis=0)
    return None


def transcribe_audio(audio_array):
    """Transcribe audio to text using OpenAI."""
    audio_bytes = audio_array.tobytes()
    audio_file = io.BytesIO(audio_bytes)
    audio_file.name = "user_input.wav"
    
    transcript = openai_client.audio.transcriptions.create(
        model="gpt-4o-transcribe",
        file=audio_file,
        response_format="text"
    )
    
    print(f"You said: {transcript.text}")
    return transcript.text

print("✅ Audio functions ready")

✅ Audio functions ready


## Step 5: Voice Assistant Function

This is the main function that ties everything together - it connects to ZeroEntropy via MCP, sets up the voice pipeline, and handles the conversation loop.

In [ ]:
async def run_voice_assistant():
    """Main voice assistant loop."""
    print("\n🎙️ Voice Assistant")
    print("📝 Instructions:")
    print("   • Press ENTER to start recording")
    print("   • Press ENTER again to stop recording and send")
    print("   • Press ENTER during AI response to interrupt")
    print("   • Press Ctrl+C to exit")
    print("-" * 50)
    
    # Setup MCP connection
    async with MCPServerSse(
        name="ZeroEntropy",
        params={
            "url": MCP_URL,
            "headers": {
                "Authorization": f"Bearer {ZEROENTROPY_API_KEY}",
                "X-Collection-Name": COLLECTION_NAME,
            }
        },
        client_session_timeout_seconds=10.0,
    ) as search_server:
        
        # Create agent
        agent = Agent(
            name="ZeroEntropyVoiceAgent",
            instructions=SYSTEM_PROMPT,
            mcp_servers=[search_server],
            model="gpt-4.1-mini",
        )
        
        # Setup voice pipeline
        tts_settings = TTSModelSettings(instructions=TTS_PROMPT)
        voice_config = VoicePipelineConfig(tts_settings=tts_settings)
        workflow = SingleAgentVoiceWorkflow(agent)
        pipeline = VoicePipeline(workflow=workflow, config=voice_config)
        
        # Setup audio output
        output_stream = sd.OutputStream(
            samplerate=SAMPLE_RATE,
            channels=CHANNELS,
            dtype="int16"
        )
        output_stream.start()
        
        try:
            while True:
                # Record user input
                audio_array = record_enter_to_talk()
                if audio_array is None:
                    continue
                
                # Process request
                user_input = AudioInput(buffer=audio_array)
                result = await pipeline.run(user_input)
                
                # Setup interruption detection
                stop_playback = threading.Event()
                
                def monitor_interrupt():
                    """Monitor for Enter key press to interrupt response."""
                    try:
                        while not stop_playback.is_set():
                            if hasattr(select, 'select'):
                                ready, _, _ = select.select([sys.stdin], [], [], 0.1)
                                if ready:
                                    sys.stdin.readline()
                                    stop_playback.set()
                                    return
                            else:
                                time.sleep(0.1)
                    except:
                        pass
                
                # Start interrupt monitoring thread
                interrupt_thread = threading.Thread(target=monitor_interrupt, daemon=True)
                interrupt_thread.start()
                
                # Stream response
                print("🤖 Assistant: ", end="", flush=True)
                try:
                    async for event in result.stream():
                        if stop_playback.is_set():
                            print("\n[Interrupted]")
                            break
                        if event.type == "voice_stream_event_audio":
                            output_stream.write(event.data)
                        elif event.type == "voice_stream_event_transcript":
                            print(event.text, end="", flush=True)
                except Exception:
                    pass
                
                # Signal the interrupt thread to stop
                stop_playback.set()
                print("\n" + "-" * 30)
                
        except KeyboardInterrupt:
            print("\n👋 Goodbye!")
        finally:
            output_stream.stop()
            output_stream.close()

print("✅ Voice assistant function ready")

## Step 6: Run the Voice Assistant

Ready to start! This will check your setup and launch the voice assistant.

In [ ]:
# Enable nested asyncio for Jupyter
import nest_asyncio
nest_asyncio.apply()

# Check API keys first
if not ZEROENTROPY_API_KEY or not OPENAI_API_KEY:
    print("❌ Error: Missing API keys!")
    print("Please set ZEROENTROPY_API_KEY and OPENAI_API_KEY in your .env file")
else:
    print("✅ API keys found")
    print("🔧 Testing audio device...")
    
    try:
        # Quick audio test
        test_audio = sd.rec(int(0.1 * SAMPLE_RATE), samplerate=SAMPLE_RATE, channels=CHANNELS, dtype="int16")
        sd.wait()
        print("✅ Audio device working")
        print("\n🚀 Starting Voice Assistant...")
        print("💡 Remember: Use ENTER key to control recording")
        
        # Run the assistant
        await run_voice_assistant()
        
    except Exception as e:
        print(f"❌ Audio setup failed: {e}")
        print("Please check your microphone and audio permissions")


## Usage Notes

### How to Use:
1. **Start Recording**: Press ENTER
2. **Stop Recording**: Press ENTER again
3. **Interrupt Assistant**: Press ENTER while AI is speaking
4. **Exit**: Press Ctrl+C

### Example Questions:
- "Tell me about ZeroEntropy?"
- "What YC companies work on AI in San Francisco?"
- "Find me companies in healthcare"
- "What does Stripe do?"

### Troubleshooting:
- **No audio**: Check microphone permissions and connection
- **API errors**: Verify your API keys in the `.env` file
- **Interruption not working**: Make sure your terminal supports input during execution

### Key Features:
- ✅ **Interrupt capability** - Stop AI responses anytime
- ✅ **Real-time search** - Connects to live YC company database
- ✅ **Voice-to-voice** - Full speech input and output